In [ ]:
#quotas restant api
import requests
import time
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

API_KEY = ="" # Assumes your API key is stored in an environment variable

def get_remaining_quotas(api_key):
    url = "https://api.the-odds-api.com/v4/sports"
    params = {"apiKey": api_key}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        # Quota info is in response headers
        quota_used = response.headers.get("x-requests-used")
        quota_remaining = response.headers.get("x-requests-remaining")
        return quota_used, quota_remaining
    else:
        raise Exception(f"API request failed: {response.status_code} {response.text}")

if API_KEY:
    used, remaining = get_remaining_quotas(API_KEY)
    print(f"API quota used: {used}, remaining: {remaining}")
    with open("./data/quotas.txt", "a") as f:
        f.write(f"API quota used: {used}, remaining: {remaining} at {time.ctime()}\n")
else:
    print("ODDS_API_KEY environment variable not set.")

ODDS_API_KEY environment variable not set.


In [3]:
"""
Script pour récupérer et filtrer les cotes avec The Odds API (v4).

Ce script montre comment obtenir les cotes 1N2 (domicile/nul/extérieur)
pour un ou plusieurs sports, identifier les événements où les trois issues
ont des cotes proches d'une valeur cible, et assembler les résultats dans un DataFrame pandas.

Principales fonctionnalités :

    • Utilise la clé API fournie pour authentifier les requêtes vers The Odds API.
    • Demande les cotes décimales pour le marché h2h (1N2) et inclut les liens profonds
        (`includeLinks=true`) et les identifiants de source (`includeSids=true`).
    • Extrait le meilleur prix disponible pour chaque issue par bookmaker et
        sélectionne le lien le plus profond dans la hiérarchie issue → marché →
        bookmaker → événement, comme recommandé par la documentation de The Odds API.
    • Filtre les bookmakers dont les cotes pour les trois issues sont toutes dans
        une tolérance configurable autour d'une valeur cible (par défaut 3.0 ± 0.6) et trie
        les résultats par le produit des trois cotes.

Utilisation :

    Définissez la variable `API_KEY` ci-dessous avec votre clé Odds API valide. Puis lancez :

            python3 odds_filter_script.py

    Par défaut, le script interroge la Premier League anglaise (clé de sport
    'soccer_epl'). Vous pouvez changer `SPORT_KEYS` pour une liste d'autres sports
    (ex : ['americanfootball_nfl', 'basketball_nba']) ou utiliser l'endpoint
    '/v4/sports' pour découvrir les clés disponibles.

Dépendances :

    pip install requests pandas

Note :
    Ce script est prévu pour une exécution locale. Les appels réseau sont désactivés
    dans cet environnement de chat, donc il ne fonctionnera pas ici.
    Enregistrez-le sur votre machine et exécutez-le avec un accès internet.
"""

import os
from typing import Dict, List, Optional, Any, Tuple

import requests
import pandas as pd

# ============================ Configuration ============================

# Remplacez ceci par votre propre clé API. Ne partagez jamais de clé réelle publiquement.
API_KEY: str = "5bb24e9151caf14c896925a570220a78"

# Liste des clés de sport à interroger. 'soccer_epl' pour la Premier League,
# 'upcoming' pour un mélange de sports, ou appelez /v4/sports pour en découvrir d'autres.
SPORT_KEYS: List[str] = ["upcoming"]

# Cote cible et tolérance. Le script sélectionne les événements où les cotes domicile,
# nul et extérieur sont toutes dans [TARGET - TOL, TARGET + TOL].
TARGET: float = 3.0
TOL: float = 0.6

# Codes régionaux pour les bookmakers. 'eu' retourne les bookmakers européens.
REGION: str = "eu"

# =========================== Fonctions Utilitaires ===========================

def fetch_odds(
        api_key: str,
        sport_key: str,
        region: str = REGION,
        markets: str = "h2h",
        odds_format: str = "decimal",
        include_links: bool = True,
        include_sids: bool = True,
) -> List[Dict[str, Any]]:
        """Récupère les cotes pour un sport donné via The Odds API.

        Args:
                api_key: Votre clé Odds API.
                sport_key: Identifiant du sport (ex : 'soccer_epl').
                region: Code(s) régionaux comme 'eu', 'us', 'uk'. Séparez par des virgules pour plusieurs.
                markets: Marché ; 'h2h' retourne les cotes 1N2 pour le foot.
                odds_format: Format des cotes ('decimal' ou 'american').
                include_links: Inclure les liens profonds (ajoute includeLinks=true).
                include_sids: Inclure les IDs de source (ajoute includeSids=true).

        Returns:
                Une liste de dictionnaires d'événements retournés par l'API.
        """
        url = f"https://api.the-odds-api.com/v4/sports/{sport_key}/odds/"
        params: Dict[str, Any] = {
                "apiKey": api_key,
                "regions": region,
                "markets": markets,
                "oddsFormat": odds_format,
        }
        if include_links:
                params["includeLinks"] = "true"
        if include_sids:
                params["includeSids"] = "true"
        response = requests.get(url, params=params, timeout=30)
        if response.status_code == 404:
                # Sport non disponible ou hors saison
                return []
        response.raise_for_status()
        return response.json()

def _deep_link(event: Dict, bookmaker: Dict, market: Dict, outcome: Dict) -> Optional[str]:
        """Retourne le lien le plus profond selon la hiérarchie de l'API.

        Ordre recommandé : outcome.link → market.link → bookmaker.link → event.link

        Args:
                event: Dictionnaire d'événement pouvant contenir un 'link'.
                bookmaker: Dictionnaire bookmaker pouvant contenir un 'link'.
                market: Dictionnaire marché pouvant contenir un 'link'.
                outcome: Dictionnaire issue pouvant contenir un 'link'.

        Returns:
                Le premier lien non vide trouvé ou None.
        """
        return (
                outcome.get("link")
                or market.get("link")
                or bookmaker.get("link")
                or event.get("link")
        )

def extract_bookmaker_odds(event: Dict) -> List[Dict[str, Any]]:
        """Extrait les meilleures cotes domicile/nul/extérieur et liens par bookmaker pour un événement.

        Args:
                event: Dictionnaire d'événement de l'API, contenant bookmakers et marchés.

        Returns:
                Une liste où chaque élément correspond à un bookmaker proposant les trois issues.
                Chaque élément contient les cotes et liens profonds.
        """
        results: List[Dict[str, Any]] = []
        home_team = event.get("home_team")
        away_team = event.get("away_team")
        for bookmaker in event.get("bookmakers", []):
                book_key = bookmaker.get("key")
                book_title = bookmaker.get("title", book_key)
                for market in bookmaker.get("markets", []):
                        if market.get("key") != "h2h":
                                continue
                        odds_map: Dict[str, float] = {}
                        link_map: Dict[str, Optional[str]] = {}
                        for outcome in market.get("outcomes", []):
                                name = outcome.get("name", "")
                                price = outcome.get("price")
                                if name.lower() in {"draw", "tie", "egalité"}:
                                        key = "draw"
                                elif name == home_team:
                                        key = "home"
                                elif name == away_team:
                                        key = "away"
                                else:
                                        continue
                                try:
                                        price_val = float(price)
                                except (TypeError, ValueError):
                                        continue
                                odds_map[key] = price_val
                                link_map[key] = _deep_link(event, bookmaker, market, outcome)
                        if len(odds_map) == 3:
                                results.append(
                                        {
                                                "event_id": event.get("id"),
                                                "sport_key": event.get("sport_key"),
                                                "commence_time": event.get("commence_time"),
                                                "home_team": home_team,
                                                "away_team": away_team,
                                                "bookmaker": book_key,
                                                "bookmaker_title": book_title,
                                                "home_odds": odds_map["home"],
                                                "draw_odds": odds_map["draw"],
                                                "away_odds": odds_map["away"],
                                                "home_link": link_map["home"],
                                                "draw_link": link_map["draw"],
                                                "away_link": link_map["away"],
                                        }
                                )
        return results

def find_events_with_target_odds(
        api_key: str,
        sport_keys: List[str],
        target: float = TARGET,
        tol: float = TOL,
        region: str = REGION,
) -> pd.DataFrame:
        """Récupère et filtre les offres bookmakers proches d'une cote cible.

        Args:
                api_key: Clé Odds API.
                sport_keys: Liste des identifiants de sport à interroger.
                target: Cote souhaitée autour de laquelle les trois issues doivent se situer.
                tol: Tolérance autour de la cote cible.
                region: Code(s) régionaux pour les bookmakers (ex : 'eu').

        Returns:
                Un DataFrame pandas avec une ligne par (événement, bookmaker) répondant aux critères.
                Colonnes : équipes, cotes, liens profonds et produit des cotes pour le tri.
        """
        records: List[Dict[str, Any]] = []
        for sport_key in sport_keys:
                try:
                        events = fetch_odds(api_key, sport_key, region=region)
                except Exception as exc:
                        print(f"Erreur lors de la récupération des cotes pour {sport_key} : {exc}")
                        continue
                for event in events:
                        for rec in extract_bookmaker_odds(event):
                                if (
                                        abs(rec["home_odds"] - target) <= tol
                                        and abs(rec["draw_odds"] - target) <= tol
                                        and abs(rec["away_odds"] - target) <= tol
                                ):
                                        rec["combined_product"] = (
                                                rec["home_odds"] * rec["draw_odds"] * rec["away_odds"]
                                        )
                                        records.append(rec)
        df = pd.DataFrame(records)
        if not df.empty:
                df = df.sort_values("combined_product", ascending=False).reset_index(drop=True)
        return df

def main() -> None:
        """Point d'entrée du script. Récupère les cotes, filtre et affiche le résultat."""
        if not API_KEY:
                print("Aucune clé API n'est configurée. Veuillez définir API_KEY dans le script.")
                return
        if not SPORT_KEYS:
                print("Aucune clé de sport fournie. Spécifiez au moins un sport.")
                return
        df = find_events_with_target_odds(API_KEY, SPORT_KEYS, TARGET, TOL, REGION)
        if df.empty:
                print("Aucun événement correspondant aux critères n'a été trouvé.")
        else:
                with pd.option_context('display.max_columns', None):
                        print(df.to_string(index=False))
                df.to_csv("./data/data-paris-sportifs.csv", index=False)
                print("\nLes résultats ont été enregistrés dans le fichier 'data-paris-sportifs.csv'.")

if __name__ == "__main__":
        main()
 

                        event_id                 sport_key        commence_time        home_team      away_team       bookmaker    bookmaker_title  home_odds  draw_odds  away_odds                                                                                                                                        home_link                                                                                                                                        draw_link                                                                                                                                        away_link  combined_product
9c2eb7c53fbd864220cd1181ba14fe4d      soccer_italy_serie_a 2025-10-05T13:00:00Z       Fiorentina        AS Roma   betfair_ex_eu            Betfair       3.30       3.20       2.58                                                                                https://www.betfair.com/exchange/plus/football/market/1.248073501                                           

In [7]:
data_paris_sportifs = pd.read_csv("./data/data-paris-sportifs.csv")
data_paris_sportifs

,event_id,sport_key,commence_time,home_team,away_team,bookmaker,bookmaker_title,home_odds,draw_odds,away_odds,home_link,draw_link,away_link,combined_product
0,9c2eb7c53fbd864220cd1181ba14fe4d,soccer_italy_serie_a,2025-10-05T13:00:00Z,Fiorentina,AS Roma,betfair_ex_eu,Betfair,3.30,3.20,2.58,https://www.betfair.com/exchange/plus/football...,https://www.betfair.com/exchange/plus/football...,https://www.betfair.com/exchange/plus/football...,27.244800
1,9c2eb7c53fbd864220cd1181ba14fe4d,soccer_italy_serie_a,2025-10-05T13:00:00Z,Fiorentina,AS Roma,matchbook,Matchbook,3.30,3.20,2.56,https://www.matchbook.com/events/soccer/italy/...,https://www.matchbook.com/events/soccer/italy/...,https://www.matchbook.com/events/soccer/italy/...,27.033600
2,1a7d9a230fc5e77b5fbf420993a8eb93,soccer_epl,2025-10-05T13:00:00Z,Everton,Crystal Palace,betfair_ex_eu,Betfair,2.80,3.20,2.98,https://www.betfair.com/exchange/plus/football...,https://www.betfair.com/exchange/plus/football...,https://www.betfair.com/exchange/plus/football...,26.700800
3,1a7d9a230fc5e77b5fbf420993a8eb93,soccer_epl,2025-10-05T13:00:00Z,Everton,Crystal Palace,matchbook,Matchbook,2.80,3.20,2.98,https://www.matchbook.com/events/soccer/englan...,https://www.matchbook.com/events/soccer/englan...,https://www.matchbook.com/events/soccer/englan...,26.700800
4,1a7d9a230fc5e77b5fbf420993a8eb93,soccer_epl,2025-10-05T13:00:00Z,Everton,Crystal Palace,coolbet,Coolbet,2.77,3.18,2.90,https://www.coolbet.com/en/sports/match/4420209,https://www.coolbet.com/en/sports/match/4420209,https://www.coolbet.com/en/sports/match/4420209,25.544940
5,9c2eb7c53fbd864220cd1181ba14fe4d,soccer_italy_serie_a,2025-10-05T13:00:00Z,Fiorentina,AS Roma,pinnacle,Pinnacle,3.25,3.19,2.46,https://www.pinnacle.com/en/soccer/italy-serie...,https://www.pinnacle.com/en/soccer/italy-serie...,https://www.pinnacle.com/en/soccer/italy-serie...,25.504050
6,1a7d9a230fc5e77b5fbf420993a8eb93,soccer_epl,2025-10-05T13:00:00Z,Everton,Crystal Palace,unibet_se,Unibet (SE),2.80,3.25,2.80,https://www.unibet.se/betting/sports/event/102...,https://www.unibet.se/betting/sports/event/102...,https://www.unibet.se/betting/sports/event/102...,25.480000
7,1a7d9a230fc5e77b5fbf420993a8eb93,soccer_epl,2025-10-05T13:00:00Z,Everton,Crystal Palace,unibet_nl,Unibet (NL),2.80,3.25,2.80,NaN,NaN,NaN,25.480000
8,6da729437080e553fb91c8eb73ad3ae8,soccer_austria_bundesliga,2025-10-05T12:31:00Z,Rheindorf Altach,Sturm Graz,betfair_ex_eu,Betfair,3.15,2.82,2.84,https://www.betfair.com/exchange/plus/football...,https://www.betfair.com/exchange/plus/football...,https://www.betfair.com/exchange/plus/football...,25.227720
9,9c2eb7c53fbd864220cd1181ba14fe4d,soccer_italy_serie_a,2025-10-05T13:00:00Z,Fiorentina,AS Roma,coolbet,Coolbet,3.20,3.15,2.50,https://www.coolbet.com/en/sports/match/4424966,https://www.coolbet.com/en/sports/match/4424966,https://www.coolbet.com/en/sports/match/4424966,25.200000


In [9]:
# Filtrer le dataframe pour afficher uniquement les bookmakers français (contenant '(FR)' dans 'bookmaker_title')
df_paris_sportifs_fr = data_paris_sportifs[data_paris_sportifs['bookmaker_title'].str.contains(r'\(FR\)', na=False)]
df_paris_sportifs_fr

,event_id,sport_key,commence_time,home_team,away_team,bookmaker,bookmaker_title,home_odds,draw_odds,away_odds,home_link,draw_link,away_link,combined_product
48,1a7d9a230fc5e77b5fbf420993a8eb93,soccer_epl,2025-10-05T13:00:00Z,Everton,Crystal Palace,winamax_fr,Winamax (FR),2.55,3.10,2.65,https://www.winamax.fr/en/sports-betting/match...,https://www.winamax.fr/en/sports-betting/match...,https://www.winamax.fr/en/sports-betting/match...,20.94825
49,1a7d9a230fc5e77b5fbf420993a8eb93,soccer_epl,2025-10-05T13:00:00Z,Everton,Crystal Palace,betclic_fr,Betclic (FR),2.43,3.08,2.75,https://www.betclic.fr/football-sfootball/angl...,https://www.betclic.fr/football-sfootball/angl...,https://www.betclic.fr/football-sfootball/angl...,20.58210
50,1a7d9a230fc5e77b5fbf420993a8eb93,soccer_epl,2025-10-05T13:00:00Z,Everton,Crystal Palace,unibet_fr,Unibet (FR),2.46,3.05,2.72,https://www.unibet.fr/sport/football/event/eve...,https://www.unibet.fr/sport/football/event/eve...,https://www.unibet.fr/sport/football/event/eve...,20.40816
51,1a7d9a230fc5e77b5fbf420993a8eb93,soccer_epl,2025-10-05T13:00:00Z,Everton,Crystal Palace,parionssport_fr,Parions Sport (FR),2.55,2.95,2.70,https://www.enligne.parionssport.fdj.fr/paris-...,https://www.enligne.parionssport.fdj.fr/paris-...,https://www.enligne.parionssport.fdj.fr/paris-...,20.31075
56,6da729437080e553fb91c8eb73ad3ae8,soccer_austria_bundesliga,2025-10-05T12:31:00Z,Rheindorf Altach,Sturm Graz,winamax_fr,Winamax (FR),2.65,2.55,2.50,https://www.winamax.fr/en/sports-betting/match...,https://www.winamax.fr/en/sports-betting/match...,https://www.winamax.fr/en/sports-betting/match...,16.89375
57,6da729437080e553fb91c8eb73ad3ae8,soccer_austria_bundesliga,2025-10-05T12:31:00Z,Rheindorf Altach,Sturm Graz,betclic_fr,Betclic (FR),2.65,2.52,2.47,https://www.betclic.fr/football-sfootball/autr...,https://www.betclic.fr/football-sfootball/autr...,https://www.betclic.fr/football-sfootball/autr...,16.49466
